In [124]:
# Instala o PySpark na máquina virtual
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [125]:
# Importa a sessão Spark e os SparkTypes
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, DoubleType
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [126]:
# Cria a sessão Spark (lembrar de fechar a sessão depois)
spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Challenge Semana 01") \
    .getOrCreate()

In [127]:
# Cria um dataset Spark com as informações brutas do arquivo json
dataset_imoveis_bruto = spark.read.json('/content/drive/MyDrive/Challenge Data Science/dataset_bruto.json')

In [128]:
# Conta a quantidade de dados no json
dataset_imoveis_bruto.count()
dataset_imoveis_bruto.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

In [129]:
# Cria colunas para cada um dos dados contidos em anuncio
dataset_anuncio = dataset_imoveis_bruto.select('anuncio.*')
dataset_anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (c

In [130]:
# Transforma as colunas com unidades diferenciadas em suas unidades corretas
dataset_anuncio = dataset_anuncio\
  .withColumn('area_total',dataset_anuncio['area_total'][0].cast(IntegerType()))\
  .withColumn('area_util',dataset_anuncio['area_util'][0].cast(IntegerType()))\
  .withColumn('banheiros',dataset_anuncio['banheiros'][0])\
  .withColumn('quartos',dataset_anuncio['quartos'][0])\
  .withColumn('suites',dataset_anuncio['suites'][0])\
  .withColumn('vaga',dataset_anuncio['vaga'][0])\

In [131]:
# Realiza contagem por tipo anuncio
dataset_anuncio\
  .select('tipo_anuncio')\
  .groupBy('tipo_anuncio')\
  .count()\
  .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|88827|
|  Lançamento|  256|
+------------+-----+



In [132]:
# Filtra somente por imoveis usados
dataset_anuncio = dataset_anuncio\
  .select('*')\
  .where('tipo_anuncio=="Usado"')

In [133]:
# Realiza contagem por tipo anuncio após filtro
dataset_anuncio\
  .select('tipo_anuncio')\
  .groupBy('tipo_anuncio')\
  .count()\
  .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|88827|
+------------+-----+



In [134]:
dataset_anuncio.show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|      null|       16|        0|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|      0|     0|       Usado|      Outros|  Comercial|   1|[{260, 107, Venda...|
|    0|      null|       14|        0|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|      0|  null|       Usado|      Outros|  Comercial|   0|[{260, 107, Venda...|
|    0|      1026|     1026|        0|                  []|{Maria da Graça, ...|1fb030a5-9e3e-4

In [135]:
# Realiza contagem por tipo anuncio
dataset_anuncio\
  .select('tipo_uso')\
  .groupBy('tipo_uso')\
  .count()\
  .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|  Comercial| 4539|
|Residencial|84288|
+-----------+-----+



In [136]:
# Filtra somente por imoveis residenciais
dataset_anuncio = dataset_anuncio\
  .select('*')\
  .where('tipo_uso=="Residencial"')

In [137]:
# Realiza contagem por tipo anuncio após filtro
dataset_anuncio\
  .select('tipo_uso')\
  .groupBy('tipo_uso')\
  .count()\
  .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|Residencial|84288|
+-----------+-----+



In [138]:
# Realiza contagem por tipo unidade
dataset_anuncio\
  .select('tipo_unidade')\
  .groupBy('tipo_unidade')\
  .count()\
  .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
|      Outros| 7502|
| Apartamento|66562|
|        Casa|10224|
+------------+-----+



In [139]:
# Filtra somente por apartamentos
dataset_anuncio = dataset_anuncio\
  .select('*')\
  .where('tipo_unidade=="Apartamento"')

In [140]:
# Realiza contagem por tipo unidade após filtro
dataset_anuncio\
  .select('tipo_unidade')\
  .groupBy('tipo_unidade')\
  .count()\
  .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
| Apartamento|66562|
+------------+-----+



In [141]:
dataset_anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- valores: array (nullable = true)
 |    |-- ele

In [142]:
# Retira as informações de endereço necessárias, assim como as de valor
dataset_anuncio_filtrado = dataset_anuncio\
  .select('andar','area_total','area_util','banheiros','caracteristicas','endereco.bairro','endereco.zona','id','quartos','suites','tipo_anuncio','tipo_unidade','tipo_uso','vaga','valores.valor')\

In [143]:
# Converte a informação de valor em inteiro
dataset_anuncio_filtrado = dataset_anuncio_filtrado\
  .withColumn('valor',dataset_anuncio_filtrado['valor'][0].cast(IntegerType()))\

In [146]:
# Cria o output num arquivo Parquet
dataset_anuncio_filtrado.write.parquet(
    path = '/content/drive/MyDrive/Challenge Data Science/Semana 1/output_parquet',
    mode = 'overwrite',
)

In [148]:
dataset_anuncio_filtrado.show(5)

+-----+----------+---------+---------+--------------------+---------+----------+--------------------+-------+------+------------+------------+-----------+----+-----+
|andar|area_total|area_util|banheiros|     caracteristicas|   bairro|      zona|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|valor|
+-----+----------+---------+---------+--------------------+---------+----------+--------------------+-------+------+------------+------------+-----------+----+-----+
|    3|        43|       43|        1|[Academia, Churra...|Paciência|Zona Oeste|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|15000|
|    2|        42|       42|        1|[Churrasqueira, P...|Paciência|Zona Oeste|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|15000|
|    1|        41|       41|        1|[Portaria 24h, Co...|Guaratiba|Zona Oeste|18d22cbe-1b86-476...|      2|  null|       Usado| Apartamento|Residencial|   1|20000|
|   

In [149]:
# Cria o output num arquivo csv
#dataset_anuncio_filtrado.write.csv(
#    path = '/content/drive/MyDrive/Challenge Data Science/Semana 1/output_csv',
#    mode = 'overwrite',
#    sep = ';',
#    header = True,
#)

In [153]:
# Lendo o arquivo Parquet para ter certeza que foi tudo certo

dataset_parquet = spark.read.parquet('/content/drive/MyDrive/Challenge Data Science/Semana 1/output_parquet/part-00000-2ef23170-a88c-4ed8-a077-49f636575fac-c000.snappy.parquet')
dataset_parquet.show()

+-----+----------+---------+---------+--------------------+--------------------+----------+--------------------+-------+------+------------+------------+-----------+----+-----+
|andar|area_total|area_util|banheiros|     caracteristicas|              bairro|      zona|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|valor|
+-----+----------+---------+---------+--------------------+--------------------+----------+--------------------+-------+------+------------+------------+-----------+----+-----+
|    3|        43|       43|        1|[Academia, Churra...|           Paciência|Zona Oeste|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|15000|
|    2|        42|       42|        1|[Churrasqueira, P...|           Paciência|Zona Oeste|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|15000|
|    1|        41|       41|        1|[Portaria 24h, Co...|           Guaratiba|Zona Oeste|18d22cbe-1b86-476...|   

In [154]:
# Encerra a sessão Spark
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x7f76db03ba50>>